In [159]:
%run ../paths.py

In [160]:
import pandas as pd
import numpy as np
import os
import re
import operator
import gc
import time
import matplotlib.pyplot as plt
import warnings
import gensim
import numpy as np
warnings.filterwarnings('ignore') 
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
from nltk.stem.wordnet import WordNetLemmatizer 
import pyLDAvis.gensim
%matplotlib inline
from sklearn import model_selection
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm

In [189]:
train = pd.read_csv(CONST().CLEANED_TRAIN)
test = pd.read_csv(CONST().CLEANED_TEST)

In [162]:
%run ../src/features/vectorization.py

In [163]:
x,y= ngram_vectorize(CountVectorizer,train["tweet"],np.asarray(train["class"]),test["tweet"],20000)



In [191]:


def preprocess(comment):
    """
    Function to build tokenized texts from input comment
    """
    return gensim.utils.simple_preprocess(comment, deacc=True, min_len=2)




In [192]:

#tokenize the comments
train_text=train.tweet.apply(lambda x: preprocess(x))
test_text=test.tweet.apply(lambda x: preprocess(x))
all_text=train_text.append(test_text)





In [201]:
print("Total number of comments:",len(all_text))
print("Before preprocessing:",train.tweet.iloc[6])
print("After preprocessing:",all_text.iloc[6])


Total number of comments: 53963
Before preprocessing: hey someone else suppos lifestyle also happy fathers day
After preprocessing: ['hey', 'someone', 'else', 'suppos', 'lifestyle', 'also', 'happy', 'fathers', 'day']


In [194]:
#Phrases help us group together bigrams :  new + york --> new_york
bigram = gensim.models.Phrases(all_text)

In [200]:
bigram[all_text.iloc[6]]

['hey', 'someone_else', 'suppos', 'lifestyle', 'also', 'happy_fathers', 'day']

In [174]:
lem = WordNetLemmatizer()

In [202]:
def clean(word_list):
    """
    Function to clean the pre-processed word lists 
    
    Following transformations will be done
    1) Stop words removal from the nltk stopword list
    2) Bigram collation (Finding common bigrams and grouping them together using gensim.models.phrases)
    3) Lemmatization (Converting word to its root form : babies --> baby ; children --> child)
    """
    #remove stop words
    clean_words = [w for w in word_list if not w in ["will","it","they"]]
    #collect bigrams
    clean_words = bigram[clean_words]
    #Lemmatize
    clean_words=[lem.lemmatize(word, "v") for word in clean_words]
    return(clean_words)

In [203]:
all_text=all_text.apply(lambda x:clean(x))

In [204]:
#create the dictionary
dictionary = Dictionary(all_text)
print("There are",len(dictionary),"number of words in the final dictionary")


There are 27525 number of words in the final dictionary


In [205]:
corpus = [dictionary.doc2bow(text) for text in all_text]


In [207]:
print(dictionary.doc2bow(all_text.iloc[6]))
print("Wordlist from the sentence:",all_text.iloc[6])

[(16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)]
Wordlist from the sentence: ['hey', 'someone_else', 'suppos', 'lifestyle', 'also', 'happy_fathers', 'day']


In [208]:
ldamodel = LdaModel(corpus=corpus, num_topics=15, id2word=dictionary)

In [209]:
pyLDAvis.enable_notebook()

In [210]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.214597 -0.058918       1        1  12.554070
10    -0.114518  0.084482       2        1   9.178061
9     -0.203896 -0.088402       3        1   7.903524
14    -0.008565  0.172015       4        1   7.695498
6      0.106385  0.166200       5        1   7.093433
12     0.127471 -0.044454       6        1   6.647624
1     -0.141923 -0.108276       7        1   6.614484
8     -0.032734  0.047923       8        1   6.508769
4     -0.085948 -0.054313       9        1   5.887096
7      0.012474  0.125456      10        1   5.845425
3      0.059035 -0.007726      11        1   5.079364
5      0.041723 -0.125429      12        1   5.074537
0      0.003392  0.029716      13        1   4.876268
11     0.177483  0.083328      14        1   4.618692
2      0.274218 -0.221601      15        1   4.423155, topic_info=           Term          Freq         Total Category  logprob  loglift
34        bitch  12717.000000  12717.000000  Default  30.0000  30.0000
9           hoe   4003.000000   4003.000000  Default  29.0000  29.0000
125   face_with   1853.000000   1853.000000  Default  28.0000  28.0000
131    tears_of   1763.000000   1763.000000  Default  27.0000  27.0000
128         joy   1488.000000   1488.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
896      listen    170.826065    201.515167  Topic15  -4.5564   2.9531
638       thank    422.899475    733.670532  Topic15  -3.6499   2.5674
181     finally    206.016769    305.934052  Topic15  -4.3691   2.7229
689       again    247.168167    458.604553  Topic15  -4.1870   2.5002
1754      since    139.936310    278.789856  Topic15  -4.7559   2.4290

[867 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
178      10  0.991810        able
3745      3  0.983061  absolutely
2715      7  0.986485      accept
251       2  0.991820     account
1642     10  0.981073      across
...     ...       ...         ...
524       8  0.008186       young
524      11  0.012280       young
524      12  0.970085       young
313       8  0.005898    yourself
313       9  0.990832    yourself

[2198 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 11, 10, 15, 7, 13, 2, 9, 5, 8, 4, 6, 1, 12, 3])

### Chart Description:

The Left side shows the multi-dimensional "word-space" superimposed on two "Principal components" and the relative positions of all the topics.

The size of the circle represents what % of the corpus it contains.

The right side shows the word frequencies within the topic and in the whole corpus.

In [211]:
topic_probability_matix = ldamodel[corpus]

In [216]:
ldatopics = ldamodel.show_topics(formatted=False)


In [217]:
ldatopics

[(7,
  [('play', 0.036803655),
   ('call', 0.027815921),
   ('little', 0.026552947),
   ('guy', 0.023101324),
   ('baby', 0.019505536),
   ('ready', 0.018566724),
   ('another', 0.018449564),
   ('this', 0.0164719),
   ('be', 0.015626656),
   ('oh', 0.015226309)]),
 (13,
  [('bitch', 0.1686403),
   ('fuck', 0.03820198),
   ('love', 0.027642416),
   ('not', 0.026801588),
   ('get', 0.02598447),
   ('ass', 0.025881551),
   ('shit', 0.021579145),
   ('nigga', 0.016259903),
   ('yo', 0.01513381),
   ('im', 0.014821604)]),
 (0,
  [('face', 0.034746185),
   ('tomorrow', 0.029653734),
   ('big', 0.026010776),
   ('bihday', 0.025966195),
   ('well', 0.019701017),
   ('yeah', 0.018570542),
   ('same', 0.018253202),
   ('bitch', 0.016869046),
   ('post', 0.016731063),
   ('stupid', 0.014676284)]),
 (1,
  [('hoe', 0.11246263),
   ('not', 0.025581526),
   ('get', 0.025146907),
   ('no', 0.023090286),
   ('girls', 0.018409258),
   ('right', 0.016394919),
   ('fuck', 0.01636047),
   ('did_not', 0.01

change the number of topics using HDP